<a href="https://colab.research.google.com/github/jacktheson/jacktheson/blob/main/nnfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Recipe Robot:**
Sophia, Jackson, Juancy and Mia


Generally: Use a GAN for ingredients and a RNN (LSTM) for the instructions

Steps:
  1. encode this, e.g. a vector of 0's, with 1's corresponding to the words that appear in the recipe
  2. make a GAN that generates new believable recipe lists in the same way you can generate believable faces or handwritten digits
  3. Use an RNN(LSTM) to create the recipe instructions using the generated ingredients as a seed.


In [ ]:
import csv
import re 
import tensorflow as tf
import numpy as np

`CREATING VECTORS`

In [ ]:
import csv
import re 

datafile = open('RecipeNLG_dataset.csv', 'r')
datareader = csv.reader(datafile, delimiter=',')
originalData = []
for row in datareader:
    originalData.append(row)    
#print (originalData[1])

for item in originalData[1]:
    print (item + "\n")

FileNotFoundError: ignored

In [ ]:
titleIngred = []

for row in originalData:
        titleIngred.append(row[1] + " ; " + row[6])
        
titleIngred.pop(0);
        
print(titleIngred[0:4])

In [ ]:
ingredListMajor = []

for ingred in titleIngred:
    ingredString = ingred.split(" ; ")[1]
    regex = re.compile('[^a-zA-Z| |,]')
    temp = regex.sub('', ingredString).split(", ")
    for item in temp:
        ingredListMajor.append(item.lower())
    
print(ingredListMajor[0:100])

In [ ]:
ingredients = list(set(ingredListMajor))
ingredients.pop(0)
print(ingredients)

In [ ]:
ingredientVectors = [[]]

ingredientVectors.append(ingredients)
ingredientVectors.pop(0)

for index in range(len(titleIngred)):
    counter = []
    for ingred in ingredients:
        if titleIngred[index].count(ingred)>0:
            counter.append(1)
        else:
            counter.append(0)
    ingredientVectors.append(counter)

In [ ]:
print(ingredientVectors[1])

In [ ]:
ingredientVectors2 = np.array([], ndmin=2)

np.append(ingredientVectors2, ingredients)
#ingredientVectors.pop(0)

for index in range(len(titleIngred)):
    counter = []
    for ingred in ingredients:
        if titleIngred[index].count(ingred)>0:
            counter.append(1)
        else:
            counter.append(0)
    np.append(ingredientVectors2, counter)

# **gan**

In [ ]:
import csv
import re 
import tensorflow as tf
import numpy as np

import pandas as pd
from collections import Counter

from keras.layers import Dense, Activation
from keras.models import Sequential

from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU

In [ ]:
# importing original csv file 
datafile = open('RecipeNLG_dataset.csv', 'r')
datareader = csv.reader(datafile, delimiter=',')
originalData = []
for row in datareader:
    originalData.append(row)    

for item in originalData[1]:
    print (item + "\n")

FileNotFoundError: ignored

In [ ]:
# formatting first 1,000,000 data values to just be recipe title + ingredient list, separated by ';'

titleIngred = []

for row in originalData:
#for row in originalData[:1000000]:
    titleIngred.append(row[1] + " ; " + row[6])
        
titleIngred.pop(0);
        
print(titleIngred[0:4])

In [ ]:
# list of all ingredients, repeats included

ingredListMajor = []

for ingred in titleIngred:
    ingredString = ingred.split(" ; ")[1]
    regex = re.compile('[^a-zA-Z| |,-./\s]')
    temp = regex.sub('', ingredString).split(", ")
    for item in temp:
        if item != "egg" and len(item)>2:
            ingredListMajor.append(item.lower())

print(ingredListMajor[0:100])
print("LENGTH: ", len(ingredListMajor))



In [ ]:
# list of 1,000 most common ingredients

ingredients = []

for item in Counter(ingredListMajor).most_common(1000):
    ingredients.append(item[0])
    
print(ingredients)
print("LENGTH: ", len(ingredients))

In [ ]:
# double list of zeros + ones, 1 representing ingredient is present in recipe
# if recipe has less than 5 of the most common ingredients, it is tossed

ingredientVectors = [[]]

ingredientVectors.append(ingredients)

for index in range(len(titleIngred)):
    counter = []
    for ingred in ingredients:
        if titleIngred[index].count(ingred)>0:
            counter.append(1)
        else:
            counter.append(0)
    if counter.count(1)>=5:
        ingredientVectors.append(counter)

In [ ]:
# numpy array of ingredient vectors created above 

npIngredients = np.array(ingredientVectors[2:])
print(npIngredients)
print(npIngredients.shape)

In [ ]:
# create list of ingredients + directions properly formatted
# later will turn this into a string and run through LSTM

ingredSteps = []

for line in originalData:
    new = line[3].split('", ')
    regex = re.compile('[^a-zA-Z| |, | 0-9]')

    steps = []

    for part in new:
        steps.append(regex.sub('', part))
    
    counter = 1
    stepsFinal = ""

    for part in steps:
        temp = str(counter) + ". " + part
        stepsFinal = stepsFinal + temp + "\n"
        counter=counter+1
    
    line2 = line[6]
    regex = re.compile('[^a-zA-Z| |,]')
    temp = regex.sub('', line2).split(", ")

    tempIngred = ""

    for index in range(len(temp)):
        if index==0:
            tempIngred = temp[0]
        else: tempIngred += ", " + temp[index]

    final = ("INGREDIENTS: " + tempIngred + "\n\n" + stepsFinal).rstrip(stepsFinal[-1])
    ingredSteps.append(final)

for line in ingredSteps[1:10]:
    print(line)
    print("\n\n")

In [ ]:
def build_generator(input_shape, latent_dim):
    generator = Sequential()
    generator.add(Dense(100, input_dim=latent_dim, activation = 'relu')) 
    generator.add(Dense(200, activation = 'relu'))
    generator.add(Dense(input_shape, activation = 'relu'))
    return generator
def build_discriminator(input_shape):
    discriminator = Sequential()
    #discriminator.add(Flatten(input_shape=input_shape))
    discriminator.add(Dense(100, input_dim=input_shape))
    discriminator.add(LeakyReLU(alpha=0.01))
    discriminator.add(Dense(1, activation='sigmoid'))
    return discriminator
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [ ]:
input_shape = (len(npIngredients[0]))
print(input_shape)
latent_dim = 50

discriminator = build_discriminator(input_shape)
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

discriminator.trainable = False
generator = build_generator(input_shape, latent_dim)
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

In [ ]:
storeloss = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):
    
    # Attempt to load data 
    X_train = npIngredients

    # Labels
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    # Training loop
    for iteration in range(iterations):
        
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        ingred = X_train[idx]

        z = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_ingred = generator.predict(z)
        
        d_loss_acc_real = discriminator.train_on_batch(ingred, real)
        d_loss_acc_fake = discriminator.train_on_batch(gen_ingred, fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_acc_real, d_loss_acc_fake)
        
        z = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_ingred = generator.predict(z)
        g_loss = gan.train_on_batch(z, real)

        if iteration % sample_interval == 0:

            storeloss.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration)

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                  (iteration, d_loss, 100.0 * accuracy, g_loss))

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
iterations = 10000
batch_size = 128
sample_interval = 50

train(iterations, batch_size, sample_interval)

In [ ]:
latent_space_vector = np.random.normal(0, 1, (1, latent_dim))
arr = generator.predict(latent_space_vector)[0]

# count = np.count_nonzero(arr>0.5)
# for item in np.nditer(arr):
#     if item>0.5:
#         print(item)
# print('Total occurences of ">0.5" in array: ', count)

index = np.where(arr>0.5)
if index==0:
    print("none")
for num in index[0]:
    print(ingredientVectors[1][num])

In [ ]:
latent_space_vector = np.random.normal(0, 1, (1, latent_dim))
arr = generator.predict(latent_space_vector)[0]

seed = "INGREDIENTS: "

for index in range(len(arr)):
    if arr[index]>0.5:
        if len(seed)<15:
            seed = seed + ingredientVectors[1][index]
        else: 
            seed = seed + ", " + ingredientVectors[1][index]
        
seed = seed + "\n\n1. "
        
print(seed)

# **pull data out of csv and format to list of ingredients + directions**

In [ ]:
import csv
import re 

datafile = open('RecipeNLG_dataset.csv', 'r')
datareader = csv.reader(datafile, delimiter=',')
originalData = []
for row in datareader:
    originalData.append(row)    
#print (originalData[1])

for item in originalData[1]:
    print (item + "\n")

In [ ]:
ingredSteps = []

for line in originalData:
    new = line[3].split('", ')
    regex = re.compile('[^a-zA-Z| |, | 0-9]')

    steps = []

    for part in new:
        steps.append(regex.sub('', part))
    
    counter = 1
    stepsFinal = ""

    for part in steps:
        temp = str(counter) + ". " + part
        stepsFinal = stepsFinal + temp + "\n"
        counter=counter+1
    
    line2 = line[6]
    regex = re.compile('[^a-zA-Z| |,]')
    temp = regex.sub('', line2).split(", ")

    tempIngred = ""

    for index in range(len(temp)):
        if index==0:
            tempIngred = temp[0]
        else: tempIngred += ", " + temp[index]

    final = ("INGREDIENTS: " + tempIngred + "\n\n" + stepsFinal).rstrip(stepsFinal[-1])
    ingredSteps.append(final)

for line in ingredSteps[1:10]:
    print(line)
    print("\n\n")

CODE for the RNN

In [ ]:
import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
%load_ext tensorboard
text = "\n\n".join(ingredSteps)




In [ ]:

chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 80
step = 40
sentences = []
for recipe in ingredSteps[0:200]:
    for i in range(0, len(recipe) - seqlen - 1, step):
        sentences.append(recipe[i: i + seqlen + 1])


x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1



model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    
    for diversity in [0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
           
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=250,)

CODE for running the RNN

In [ ]:
for diversity in [0.1,0.5,1]:
  # To run with GAN's output change text to seed 
    sentence = text[8:88]
    print(sentence)
    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
        
    print('\n')
    print('\n')